# Advent of Code
## Day 7: Handy Haversacks
### Part One

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

>```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
>```

These rules specify the required contents for 9 bag types. In this example, every `faded blue` bag is empty, every `vibrant plum` bag contains 11 bags (5 `faded blue` and 6 `dotted black`), and so on.

You have a `shiny gold` bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

* A `bright white` bag, which can hold your `shiny gold` bag directly.
* A `muted yellow` bag, which can hold your `shiny gold` bag directly, plus some other bags.
* A `dark orange` bag, which can hold `bright white` and `muted yellow` bags, either of which could then hold your `shiny gold` bag.
* A `light red` bag, which can hold `bright white` and `muted yellow` bags, either of which could then hold your `shiny gold` bag.

So, in this example, the number of bag colors that can eventually contain at least one `shiny gold bag` is `4`.

How many bag colors can eventually contain at least one `shiny gold` bag? (The list of rules is quite long; make sure you get all of it.)

In [1]:
# Read in the inputs
inputs = [i[:-1] for i in open("Day07_input.txt").readlines()]

In [2]:
# Let's collect all the different types of bags in a dictionary, 
# where it's value pair can be all the different types of bags it contains
bag_dict = dict()

# Now let's process it
# First, run over each line of inputs
for line in inputs:
    # Each line looks like:
    #     <adjective> <colour> bags contain <list of bags>
    # So first split on contain
    bag, contains = line.split(' contain ')
    # The bag bit looks like:
    #     <adjective> <colour> bags
    # so we can split on spaces 
    bag_descriptor = tuple(bag.split(' bags')[0].split(' '))
    
    # Initialise the dictionary that describes what is contained by bag
    contains_dict = dict()
    # If contains is the 'no other bags.' string, then don't add anything to the dictionary
    if contains == 'no other bags.':
        pass
    else:
        # Otherwise, the string looks like:
        #     <number> <adjective> <colour> bag, ...
        # so first we can split on ', '
        for item in contains.split(', '):
            # then the first bit of the string is an integer, the rest dump into other as a list
            number, *other = item.split(' ')
            # Only the first two words in other are needed (the third should be the string 'bag' or 'bags.')
            contains_dict[tuple(other[:2])] = int(number)
    # Enter that dictionary under the key for the bag
    bag_dict[bag_descriptor] = contains_dict

In [3]:
def recursive_set_search(bag_dict, key, bag_set=None):
    """
    Recursively search inside bags until you reach the bottom.
    
    Note: 
        * We know to stop searching when the size of bag_set doesn't grow any larger after searching inside the bags.
        * The total at the end contains the original_bag, so we have to subtract one if we're not including it
    :param bag_dict: dict, a dict where each key is a bag and each value is a dict describing which bags it contains
    :param key: tuple, a bag key for bag_dict
    :param bag_set: set, the set of bags that contain the original_bag (starts with only the original bag)
    :return: int, the number of bags that contain the original_bag
    """
    # Initialise by putting the key into the bag_set
    if bag_set is None:
        bag_set = {key}
        
    # Copy bag_set to avoid a RuntimeError
    bag_set_copy = bag_set.copy()
    
    # For each bag in the bag_set
    for bag in bag_set_copy:
        # and each bag inside that bag
        for outer_bag in bag_dict:
            # if the outer bag contains the bag from bag_set
            if bag in bag_dict[outer_bag]:
                # append it to the set of bags that contain the original key
                bag_set |= {outer_bag}
    
    # If bag_set hasn't gotten any bigger, there are no more bags that contain the original key
    if len(bag_set_copy) == len(bag_set):
        return len(bag_set)
    # If bag_set has gotten bigger, we now need to loop through again to check if other bags contain the new bags
    else:
        return recursive_set_search(bag_dict, key, bag_set)

In [4]:
# Set the key
key = ('shiny', 'gold')
n = recursive_set_search(bag_dict, key)
# We subtract off 1 because n includes the key
print(f"There are {n-1:,} bags that contain a {' '.join(key)} bag.")

There are 332 bags that contain a shiny gold bag.


---

### Part Two

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

* `faded blue` bags contain `0` other bags.
* `dotted black` bags contain `0` other bags.
* `vibrant plum` bags contain `11` other bags: 5 `faded blue` bags and 6 `dotted black` bags.
* `dark olive` bags contain `7` other bags: 3 `faded blue` bags and 4 `dotted black` bags.


So, a single shiny gold bag must contain 1 `dark olive` bag (and the 7 bags within it) plus 2 `vibrant plum` bags (and the 11 bags within each of those): `1 + 1*7 + 2 + 2*11` = `32` bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

>```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
>```

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single `shiny gold` bag?

---

In [5]:
def count_nested_bags(bag_dict, key):
    """
    Count the total number of bags you're packing. Note this includes the outermost bag.
    
    :param bag_dict: dict, a dict where each key is a bag and each value is a dict describing which bags it contains
    :param key: tuple, each key is a tuple containing the two descriptors (e.g. ('shiny', 'gold'))
    :return: int, the total number of bags you're packing
    """
    # Initialise total as 1 for the outer bag
    total = 1
    
    # Sum the inner bags multiplied by how many there are
    # Note we're calling this function recursively using each inner bag inside the outer bag
    for bag in bag_dict[key]:
        total += count_nested_bags(bag_dict, bag)*bag_dict[key][bag]
    
    # Return the total
    return total

In [6]:
key = ('shiny', 'gold')
total_bags = count_nested_bags(bag_dict, key)
# We subtract 1 off because we don't want to count the outermost shiny gold bag
print(f"There are a total of {total_bags - 1:,} bags contained within the {' '.join(key)} bag")

There are a total of 10,875 bags contained within the shiny gold bag


---